이 노트북 내용을 실행하기 위해서는 아래 내용을 실행해야 합니다.

```zsh
> pip install dataset seaborn ipython-sql
```



패키지를 불러옵니다.

In [31]:
import pandas as pd

# To import `iris` data
import seaborn as sns

# For DB connection
import dataset

from sqlalchemy import create_engine


%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


`ipython-sql`을 `%load_ext sql`로 불러서 사용할 수 있는데

Cell 안에서 SQL Query를 바로 사용할 수 있어서 편리합니다.

`iris` 데이터를 불러옵니다.

In [3]:
df_iris = sns.load_dataset('iris')

In [4]:
df_iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


DB 흉내를 내기 위해 `sqlite3` 엔진을 만듭니다.

In [9]:
db_connection_address = 'sqlite:///iris.db'

disk_engine = create_engine(db_connection_address)

아래와 같이 `iris` 데이터를 `sqlite3`를 이용하여 디스크에 DB 형식으로 저장합니다.

In [10]:
db_table_name = 'iris'

df_iris.to_sql(
    name=db_table_name,
    con=disk_engine,
    index=False,
)

아래와 같이 파일로 저장된 형태의 DB에 접근할 수도 있습니다.

RDS나 다른 DB 경우에는 `db_connection_address`만 바꾸면 됩니다.

`$`로 Python 변수를 넣어줄 수 있습니다.

아래에서는 DB에 INDEX 를 생성합니다.

In [13]:
%%sql $db_connection_address
CREATE INDEX species
ON iris (species);

Done.


[]

아래와 같이 DB 테이블을 바로 조회할 수 있습니다.

In [17]:
%%sql $db_connection_address
SELECT *
  FROM iris
 WHERE species = 'setosa'
 LIMIT 5

Done.


sepal_length,sepal_width,petal_length,petal_width,species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa


`dataset` 패키지를 사용하면 코드 안에서 DataFrame으로 받아서 사용하기 편합니다.

In [19]:
db_connection = dataset.connect(db_connection_address)

In [20]:
iris_table = db_connection['iris']

In [21]:
print(iris_table.columns)

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']


In [29]:
df_iris_from_db = pd.DataFrame([r for r in iris_table.all()])

In [30]:
df_iris_from_db

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
